# 텐서플로우 튜토리얼 #03-C
# 케라스 API

by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)
/ [GitHub](https://github.com/Hvass-Labs/TensorFlow-Tutorials) / [Videos on YouTube](https://www.youtube.com/playlist?list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ)

## 소개

튜토리얼 #02는 텐서플로에서 콘볼루션 신경망을 구현하는 방법을 보여 주었다. 우리는 네트워크에 레이어를 만들기 위한 몇 가지 도우미 기능을 만들었다. 복잡한 모델을 훨씬 쉽게 구현할 수 있고, 오류 위험도 낮출 수 있기 때문에 수준 높은 API를 잘 갖추는 것이 필수적이다.
TensorFlow에 사용할 수 있는 빌더 API는 FeattyTensor(튜토리얼 #03), 레이어 API(튜토리얼 #03-B), 기타 여러 가지가 있다. 그러나 그들은 결코 진정으로 완성되지 않았고 지금은 개발자들로부터 다소 버림받은 것 같다.

이 튜토리얼은 매우 우수한 문서로 이미 고도로 개발된 Keras API에 관한 것이며, 개발은 계속된다. 향후에는 케라스가 텐서플로우의 표준 API가 될 것으로 보여 다른 API 대신 사용하는 것이 좋다.

케라스의 저자는 당신이 읽어야 할 API 설계 철학에 대해 [https://blog.keras.io/user-experience-design-for-apis.html)-post](https://blog.keras.io/user-experience-design-for-apis.html)를 썼다.

## 플로우 차트

다음 도표는 아래에 구현된 컨볼루션 신경망에서 데이터가 어떻게 흐르는지 대략적으로 보여준다. 콘볼루션에 대한 자세한 설명은 자습서 #02를 참조하십시오.

각각 최대풀링을 이용한 다운샘플링(이 흐름도에는 표시되지 않음)이 뒤따르는 두 개의 콘볼루션 레이어가 있다. 그리고 소프트맥스 분류기로 끝나는 완전 연결 층이 두 개 있다.

![Flowchart](images/02_network_flowchart.png)

## Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math

케라스에서 여러 가지를 import해야 해.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Input
from tensorflow.keras.layers import Reshape, MaxPooling2D
from tensorflow.keras.layers import Conv2D, Dense, Flatten

이것은 Python 3.6 (Anaconda)과 TensorFlow 버전을 사용하여 개발되었다.

In [ ]:
tf.__version__

## 데이터 로드

MNIST 데이터 세트는 약 12MB로 주어진 경로에 위치하지 않으면 자동으로 다운로드된다.

In [ ]:
from mnist import MNIST
data = MNIST(data_dir="data/MNIST/")

MNIST 데이터 세트는 현재 로드되었으며 이미지용 70.000개의 영상과 클래스 번호로 구성되어 있다. 데이터 세트는 3개의 상호 배타적인 하위 세트로 분할된다. 이번 튜토리얼에서는 훈련과 시험 세트만 사용할 것이라고 말했다.

In [ ]:
print("Size of:")
print("- Training-set:\t\t{}".format(data.num_train))
print("- Validation-set:\t{}".format(data.num_val))
print("- Test-set:\t\t{}".format(data.num_test))

편리를 위해 데이터 치수 일부를 복사하십시오.

In [ ]:
# 이미지의 각 차원에 있는 픽셀 수.
img_size = data.img_size

# 영상이 이 길이의 1차원 배열로 저장됨.
img_size_flat = data.img_size_flat

# 배열을 재구성하는 데 사용되는 이미지의 높이와 너비가 있는 튜플.
img_shape = data.img_shape

# 배열을 재구성하는 데 사용되는 높이, 너비 및 깊이의 튜플.
# 이것은 케라스에서 재구성에 사용된다..
img_shape_full = data.img_shape_full

# 클래스 수, 10자리당 하나씩.
num_classes = data.num_classes

# 영상의 컬러 채널 수: 그레이 스케일의 경우 1채널.
num_channels = data.num_channels

### 이미지 플롯을 위한 도우미 함수

9개의 영상을 3x3 그리드에 플로팅하고 각 이미지 아래에 참 클래스와 예측 클래스를 작성하는 데 사용되는 함수

In [ ]:
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    # 3x3 하위 그림을 사용하여 그림 만들기.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # 플롯 이미지.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # 진실되고 예측된 수업을 보여라.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

    # 클래스를 x축의 레이블로 표시.
        ax.set_xlabel(xlabel)
        
    # 플롯에서 눈금 제거.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # 그림이 여러 그림으로 올바르게 표시되는지 확인
    # 단일 노트북 셀에.
    plt.show()

 ### 데이터가 올바른지 보기 위해 몇 개의 이미지를 플롯하십시오.

In [ ]:
# 테스트 세트에서 첫 번째 이미지 가져오기.
images = data.x_test[0:9]

# 해당 이미지의 실제 클래스 가져오기.
cls_true = data.y_test_cls[0:9]

# 위의 도우미 함수를 사용하여 이미지 및 레이블 그림 그리기.
plot_images(images=images, cls_true=cls_true)

### 예시 오류을 표시하는 도우미 함수

잘못 분류된 테스트 세트의 이미지 예시를 플로팅하는 함수.

In [ ]:
def plot_example_errors(cls_pred):
    # cls_pred는 다음에 대해 예측된 클래스 번호의 배열이다.
    # 테스트 세트의 모든 이미지.

    # 예측 클래스가 잘못된지 여부를 나타내는 부울 배열.
    incorrect = (cls_pred != data.y_test_cls)

    # 테스트 세트에서 이미지의 이미지를 가져오십시오.
    # 잘못 분류된.
    images = data.x_test[incorrect]
    
    # 해당 이미지의 예측 클래스 가져오기.
    cls_pred = cls_pred[incorrect]

    # 해당 이미지의 실제 클래스 가져오기.
    cls_true = data.y_test_cls[incorrect]
    
    # 처음 9개 이미지 플롯.
    plot_images(images=images[0:9],
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])

## 프리티텐서 API

자습서 #03에서 프리티텐서 API를 이용하여 콘볼루션 신경망을 구현한 방법은 이렇다. 아래의 케라스 구현과 쉽게 비교할 수 있도록 여기에 표시된다.

In [ ]:
if False:
    x_pretty = pt.wrap(x_image)

    with pt.defaults_scope(activation_fn=tf.nn.relu):
        y_pred, loss = x_pretty.\
            conv2d(kernel=5, depth=16, name='layer_conv1').\
            max_pool(kernel=2, stride=2).\
            conv2d(kernel=5, depth=36, name='layer_conv2').\
            max_pool(kernel=2, stride=2).\
            flatten().\
            fully_connected(size=128, name='layer_fc1').\
            softmax_classifier(num_classes=num_classes, labels=y_true)

## 순차 모델

케라스 API는 뉴럴 네트워크 구축의 두 가지 모드를 가지고 있다. 가장 간단한 것이 순차적으로만 레이어를 추가할 수 있는 순차적 모델이다.

In [ ]:
# Keras Sequential 모델 구축 시작.
model = Sequential()

# TensorFlow에서 feed_dict와 유사한 입력 계층 추가.,
# 입력 셰이프는 이미지 크기를 포함하는 튜플이어야 함.
model.add(InputLayer(input_shape=(img_size_flat,)))

# 입력은 784개의 요소로 이루어진 평평한 배열이다.
# 그러나 합성층은 shape (28, 28, 1) 이미지를 기대한다.
model.add(Reshape(img_shape_full))

# ReLU 활성화 및 최대 풀링을 포함한 첫 번째 합성층.
model.add(Conv2D(kernel_size=5, strides=1, filters=16, padding='same',
                 activation='relu', name='layer_conv1'))
model.add(MaxPooling2D(pool_size=2, strides=2))

# ReLU-활성화 및 최대풀이 있는 두 번째 합성층.
model.add(Conv2D(kernel_size=5, strides=1, filters=36, padding='same',
                 activation='relu', name='layer_conv2'))
model.add(MaxPooling2D(pool_size=2, strides=2))

# 콘볼루션 레이어의 4등급 출력 평탄화
# 완전히 연결된/밀도 층에 입력할 수 있는 2등급까지.
model.add(Flatten())

# ReLU 활성화로 첫 번째 완전 연결/밀도 레이어.
model.add(Dense(128, activation='relu'))

# 소프트맥스-활성화를 통한 마지막 완전 연결/밀도 레이어
# 분류에 사용하기 위해.
model.add(Dense(num_classes, activation='softmax'))

### 모델 컴파일

신경망은 이제 정의되었으며 손실 기능, 최적화 도구, 성능 지표를 추가하여 최종 확정되어야 한다. 이것을 케라스에서는 모델 "컴필레이션"라고 부른다.

우리는 문자열을 사용하여 최적기를 정의할 수도 있고, 또는 문자열을 더 많이 제어하려면 물체를 인스턴스화할 필요가 있다. 예를 들어 학습률을 설정할 수 있다.

In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(lr=1e-3)

10개 등급이 가능한 MNIST와 같은 분류 문제의 경우 `categorical_crossentropy`라는 손실 기능을 사용해야 한다. 우리가 관심을 갖는 성능 지표는 분류 정확도

In [ ]:
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### 훈련

이제 모델이 완전히 손실 기능과 최적화 도구로 정의되었으니, 우리는 그것을 훈련시킬 수 있다. 이 함수는 숫자의 광선을 취하며 주어진 배치 크기를 사용하여 주어진 수의 훈련 에대를 수행한다. 신기원은 전체 훈련 세트를 충분히 활용하는 것이다. 그래서 10세기 동안 우리는 전체 훈련 세트에서 무작위로 10번 반복할 것이다.

In [ ]:
model.fit(x=data.x_train,
          y=data.y_train,
          epochs=1, batch_size=128)



이제 그 모델이 훈련을 받았으니, 우리는 시험 세트에서 그 성능을 시험할 수 있다. 이것은 또한 입력으로 numpy-array를 사용한다.

In [ ]:
result = model.evaluate(x=data.x_test,
                        y=data.y_test)

우리는 테스트 세트에 대한 모든 성능 지표를 출력할 수 있다.

In [ ]:
for name, value in zip(model.metrics_names, result):
    print(name, value)

아니면 분류 정확도만 출력하면 되지.

In [ ]:
print("{0}: {1:.2%}".format(model.metrics_names[1], result[1]))

### 예측

새 영상에 대한 분류도 예측할 수 있다. 우리는 테스트 세트의 일부 이미지만 사용할 것이지만 당신은 자신의 이미지를 Numpy 배열로 로드하여 대신 사용할 수 있다.

In [ ]:
images = data.x_test[0:9]

이것들이 그 이미지들의 진정한 등급 번호들이다. 이것은 영상을 플로팅할 때만 사용된다.

In [ ]:
cls_true = data.y_test_cls[0:9]

예측 클래스를 One-Hot 인코딩된 어레이로 만드십시오.

In [ ]:
y_pred = model.predict(x=images)

예측된 클래스를 정수로 받아라.

In [ ]:
cls_pred = np.argmax(y_pred, axis=1)

In [ ]:
plot_images(images=images,
            cls_true=cls_true,
            cls_pred=cls_pred)

### 잘못 분류된 이미지의 예

우리는 시험 세트에서 잘못 분류된 영상의 몇 가지 예를 그릴 수 있다.

먼저 테스트 세트의 모든 이미지에 대해 예측된 클래스를 얻으십시오.

In [ ]:
y_pred = model.predict(x=data.x_test)

그러고 나면 예측된 클래스 번호를 One-Hot 인코딩된 배열에서 정수로 변환한다.

In [ ]:
cls_pred = np.argmax(y_pred, axis=1)

잘못 분류된 영상의 일부를 플롯하십시오.

In [ ]:
plot_example_errors(cls_pred)

## 펑셔널 모델

Keras API는 Functional Model을 사용하여 더 복잡한 네트워크를 구축하는 데도 사용될 수 있다. Keras API에 대한 각 호출은 자체 호출 가능한 인스턴스를 생성하고 반환하기 때문에 처음에는 약간 혼란스러워 보일 수 있다. 그것이 함수인지 물체인지는 분명하지 않지만, 우리는 그것을 마치 함수처럼 부를 수 있다. 이를 통해 시퀀셜 모델이 허용하는 것보다 더 복잡한 연산 그래프를 만들 수 있다.

In [ ]:
# TensorFlow에서 feed_dict와 유사한 입력 계층 생성.
# 입력 셰이프는 이미지 크기를 포함하는 튜플이어야 함.
inputs = Input(shape=(img_size_flat,))

# 신경망 구축에 사용되는 변수.
net = inputs

# 입력은 784개의 요소를 가진 평평한 배열의 이미지다.
# 그러나 합성층은 shape (28, 28, 1) 이미지를 기대한다
net = Reshape(img_shape_full)(net)

# ReLU 활성화 및 최대 풀링을 포함한 첫 번째 합성층.
net = Conv2D(kernel_size=5, strides=1, filters=16, padding='same',
             activation='relu', name='layer_conv1')(net)
net = MaxPooling2D(pool_size=2, strides=2)(net)

# ReLU-활성화 및 최대풀이 있는 두 번째 합성층.
net = Conv2D(kernel_size=5, strides=1, filters=36, padding='same',
             activation='relu', name='layer_conv2')(net)
net = MaxPooling2D(pool_size=2, strides=2)(net)

# 4-dim에서 2-dim으로 콘블레이어 출력 평탄화.
net = Flatten()(net)

# ReLU 활성화로 첫 번째 완전 연결/밀도 레이어.
net = Dense(128, activation='relu')(net)

# 소프트맥스-활성화를 통한 마지막 완전 연결/밀도 레이어
# 분류에 쓰일 수 있도록.
net = Dense(num_classes, activation='softmax')(net)

# 신경망의 출력.
outputs = net

### 모델 컴파일

우리는 이제 그 입력과 출력으로 모델의 구조를 정의했다. 이제 케라스 모델을 만들어 실손 기능과 최적화 도구로 컴파일해야 하기 때문에 훈련 준비가 돼 있다고 말했다.  

In [ ]:
from tensorflow.python.keras.models import Model

케라스 함수 모델의 새 인스턴스를 만드십시오. 우리가 위에서 구축한 콘볼루션 신경망의 입력과 출력을 그것에게 준다.

In [ ]:
model2 = Model(inputs=inputs, outputs=outputs)

RMSprop Optimizer를 사용하여 여러 범주에 대해 손실 기능을 사용하여 Keras 모델을 비교하십시오. 우리가 관심을 갖는 성능 지표는 분류 정확도뿐이지만 여기서 측정 지표 목록을 사용할 수 있다.

In [ ]:
model2.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

### 훈련

이 모델은 이제 정의되고 컴파일되어 위의 순차적 모델에서 사용되는 것과 동일한 `fit()` 함수를 사용하여 훈련될 수 있게 되었다. 이것은 또한 입력으로 numpy-array를 필요로 한다.

In [ ]:
model2.fit(x=data.x_train,
           y=data.y_train,
           epochs=1, batch_size=128)

### 평가

모형이 일단 훈련을 받으면 우리는 시험 세트에서 그 성능을 평가할 수 있다. 이는 순차적 모델과 같은 구문이다.

In [ ]:
result = model2.evaluate(x=data.x_test,
                         y=data.y_test)

결과는 손실-값과 모델을 컴파일할 때 정의한 모든 메트릭스를 포함하는 값 목록이다. `accuracy`은 이제 `acc`라고 불리며, 이는 작은 모순이라는 점에 유의하십시오.

In [ ]:
for name, value in zip(model2.metrics_names, result):
    print(name, value)

분류 정확도를 백분율로 출력할 수도 있다.

In [ ]:
print("{0}: {1:.2%}".format(model2.metrics_names[1], result[1]))

### 잘못 분류된 이미지의 예

우리는 시험 세트에서 잘못 분류된 영상의 몇 가지 예를 그릴 수 있다.

먼저 테스트 세트의 모든 이미지에 대해 예측된 클래스를 얻으십시오.

In [ ]:
y_pred = model2.predict(x=data.x_test)

그러고 나면 예측된 클래스 번호를 One-Hot 인코딩된 배열에서 정수로 변환한다.

In [ ]:
cls_pred = np.argmax(y_pred, axis=1)

잘못 분류된 영상의 일부를 플롯하십시오.

In [ ]:
plot_example_errors(cls_pred)

## 저장 & 로드 모델

주의: `h5py`를 설치해야 효과가 있다!

튜토리얼 #04는 네이티브 텐서플로 코드를 사용하여 모델의 무게를 저장하고 복원하는 내용이었다. 그건 정말 끔찍한 API였어! 다행히 케라스는 이것을 아주 쉽게 만든다.

여기가 케라스 모델을 저장하려는 파일 경로야.

In [ ]:
path_model = 'model.keras'

훈련된 웨이트로 케라스 모델을 구하는 것은 그래야 하는 것처럼 단 한 번의 기능 호출에 불과하다.

In [ ]:
model2.save(path_model)

기억에서 모델을 삭제하여 더 이상 사용되지 않도록 하십시오.

In [ ]:
del model2

모델을 싣기 위해 이 케라스 함수를 import해야 해.

In [ ]:
from tensorflow.python.keras.models import load_model

모델을 적재하는 것은 그래야 하는 것처럼 단 한 번의 기능 호출에 불과하다.

In [ ]:
model3 = load_model(path_model)

예: 예측을 위해 모델을 다시 사용할 수 있다. 우리는 처음 9개의 이미지를 시험 세트와 그들의 진정한 학급 번호로부터 얻는다.

In [ ]:
images = data.x_test[0:9]

In [ ]:
cls_true = data.y_test_cls[0:9]

그 후 우리는 복원된 모델을 사용하여 그러한 이미지들에 대한 클래스 배움을 예측한다.

In [ ]:
y_pred = model3.predict(x=images)

정수로 클래스 수를 얻어라.

In [ ]:
cls_pred = np.argmax(y_pred, axis=1)

정확하고 예측된 클래스 번호로 이미지를 그려라.

In [ ]:
plot_images(images=images,
            cls_pred=cls_pred,
            cls_true=cls_true)

## 도면층 가중치 및 출력 시각화

### 콘볼루션 가중치를 플로팅하는 도우미

In [ ]:
def plot_conv_weights(weights, input_channel=0):
    # 가중치에 대한 가장 낮은 값과 가장 높은 값 가져오기.
    # 이것은 전체에서 색 강도를 보정하는 데 사용된다.
    # 서로 비교될 수 있도록 하는 이미지들.
    w_min = np.min(weights)
    w_max = np.max(weights)

    # 콘볼루션층에 사용된 필터 수.
    num_filters = weights.shape[3]

    # 플롯할 그리드 수.
    # 필터 수의 반올림, 제곱근.
    num_grids = math.ceil(math.sqrt(num_filters))
    
    # 하위 그림 그리드를 사용하여 그림 작성.
    fig, axes = plt.subplots(num_grids, num_grids)

    # 모든 필터-가중치 그림 그리기.
    for i, ax in enumerate(axes.flat):
        # 유효한 필터-가중치만 표시.
        if i<num_filters:
            # 입력 채널의 i번째 필터에 대한 가중치 가져오기.
            # 형식에 대한 자세한 내용은 new_conv_layer()를 참조하십시오.
            # 4-dim 텐서의.,
            img = weights[:, :, input_channel, i]

            # 플롯 이미지.
            ax.imshow(img, vmin=w_min, vmax=w_max,
                      interpolation='nearest', cmap='seismic')
        
        # 플롯에서 눈금 제거.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # 그림이 여러 그림으로 올바르게 표시되는지 확인
    # 단일 노트북 셀에.
    plt.show()

### 레이어 얻기

케라스는 모델에서 레이어를 나열하는 간단한 방법을 가지고 있다..

In [ ]:
model3.summary()

우리가 원하는 레이어를 얻기 위해 지수를 세는 겁니다.

입력층에는 지수 0이 있다.

In [ ]:
layer_input = model3.layers[0]

첫 번째 경련층에는 지수 2가 있다.

In [ ]:
layer_conv1 = model3.layers[2]
layer_conv1

제2의 콘볼루션 층에는 지수 4가 있다.

In [ ]:
layer_conv2 = model3.layers[4]

### 콘볼루션 가중치

이제 층이 생겼으니 그들의 무게를 쉽게 얻을 수 있다.

In [ ]:
weights_conv1 = layer_conv1.get_weights()[0]

이것은 우리에게 4단계 텐서(tensor)를 준다.

In [ ]:
weights_conv1.shape

위쪽의 도우미 함수를 사용하여 가중치를 플롯하십시오.

In [ ]:
plot_conv_weights(weights=weights_conv1, input_channel=0)

우리는 또한 두 번째 콘볼루션 층에 대한 가중치를 구해서 그것들을 플롯할 수 있다.

In [ ]:
weights_conv2 = layer_conv2.get_weights()[0]

In [ ]:
plot_conv_weights(weights=weights_conv2, input_channel=0)

### 콘볼루션 층의 출력을 플로팅하기 위한 도우미 함수

In [ ]:
def plot_conv_output(values):
    # 콘볼루션층에 사용된 필터 수.,
    num_filters = values.shape[3]

    # 플롯할 그리드 수.
    # 필터 수의 반올림, 제곱근.
    num_grids = math.ceil(math.sqrt(num_filters))
    
    # 하위 그림 그리드를 사용하여 그림 작성.
    fig, axes = plt.subplots(num_grids, num_grids)

    # 모든 필터의 출력 영상 플롯.
    for i, ax in enumerate(axes.flat):
        # 유효한 필터에 대한 영상만 표시.
        if i<num_filters:
            # i번째 필터 사용의 출력 이미지 가져오기.
            img = values[0, :, :, i]

            # 플롯 이미지.
            ax.imshow(img, interpolation='nearest', cmap='binary')
        
        # 플롯에서 누금 제거.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # 그림이 여러 그림으로 올바르게 표시되는지 확인
    # 단일 노트북 셀에.
    plt.show()

### 입력 이미지

싱글 이미지를 그리기 위한 도우미 함수

In [ ]:
def plot_image(image):
    plt.imshow(image.reshape(img_shape),
               interpolation='nearest',
               cmap='binary')

    plt.show()

아래 예제로 사용할 테스트 세트의 이미지를 플롯하십시오.

In [ ]:
image1 = data.x_test[0]
plot_image(image1)

### 콘불루션 층의 출력

콘볼루션 레이어의 출력을 보여주기 위해 우리는 원래 모델과 동일한 입력을 사용하여 또 다른 기능 모델을 만들 수 있지만, 그 출력은 현재 우리가 관심 있는 콘볼루션 레이어로부터 가져간다.

In [ ]:
output_conv2 = Model(inputs=layer_input.input,
                     outputs=layer_conv2.output)

이것은 우리가 전형적인 케라스 함수를 부를 수 있는 새로운 모델-객체를 만든다. 난막층의 출력을 얻기 위해 우리는 입력 이미지로 `predict()`함수라고 부른다.

In [ ]:
layer_output2 = output_conv2.predict(np.array([image1]))
layer_output2.shape

그러면 우리는 36개 채널 모두에 대해 영상을 그릴 수 있다.

In [ ]:
plot_conv_output(values=layer_output2)

## 결론

이번 튜토리얼에서는 텐서플로우에서 콘볼루션 신경망을 쉽게 구축할 수 있도록 이른바 *Keras API*를 사용하는 방법을 보여 주었다. 케라스는 텐서플로우를 위한 가장 완벽하고 가장 잘 설계된 API이다.

이 튜토리얼은 케라스를 사용하여 모델을 저장 및 적재하는 방법과 함께 경련 층의 가중치와 출력을 얻는 방법도 보여주었다.

이미 매우 우수하고 지속적으로 개선되고 있는 단순한 이유로 케라스가 향후 텐서플로우의 표준 API가 될 것 같다. 그러니 케라스를 이용하는 것이 좋다.

## 연습

텐서플로우(TensorFlow)로 여러분의 실력을 향상시키는 데 도움이 될 수 있는 몇 가지 운동 제안사항들이다. 텐서플로우를 제대로 사용하는 방법을 익히기 위해서는 텐서플로우를 직접 체험해 보는 것이 중요하다.,

변경하기 전에 이 노트북을 백업하십시오.

* 더 많은 시대를 위해 훈련하라. 분류 정확도를 향상시키는가?
* 일부 레이어에 대해 활성화 기능을 sigmoid로 변경하십시오.
* 모든 계층의 활성화 기능을 간단히 변경할 수 있는 방법을 찾을 수 있는가?
* conv-layer 대신 최대 풀링 레이어의 출력을 플롯하십시오.
* 2x2 최대 풀링 레이어를 경련 레이어의 stride=2로 교체한다. 분류 정확도에 차이가 있는가? 반복해서 최적화하면 어떨까? 차이가 무작위인데 정말 차이가 있다면 어떻게 측정하시겠습니까? 콘볼루션에서 최대 풀링 대 보폭을 사용하는 것의 장단점은 무엇인가?
* 커널, 깊이, 크기 등 레이어에 대한 파라미터를 변경한다. 시간 사용과 분류 정확도의 차이는 무엇인가?
* 일부 경련성 및 완전 연결성 레이어를 추가 및 제거하십시오.
* 여전히 성능이 좋은 가장 단순한 네트워크는 무엇인가?
* Functional Model을 변경하여 고밀도 레이어 안으로 들어가기 전에 기존 콘블레이어와 병렬로 연결되는 또 다른 콘볼루션 레이어를 갖도록 하십시오.
* Functional Model을 변경하여 예측 클래스를 One-Hot 인코딩된 어레이와 정수로 출력하여 그 후에 `numpy.argmax()`를 사용할 필요가 없도록 하십시오.
* 이 소스 코드를 너무 많이 보지 말고 직접 프로그램을 리메이크하십시오.
* 그 프로그램이 어떻게 작동하는지 친구에게 설명하라.

## 라이센스(MIT)

저작권 (c) 2016 by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)

이 소프트웨어의 사용, 복사, 수정, 병합, 게시, 배포, 하위 라이선스 및/또는 판매 권한을 포함하여 제한 없이 본 소프트웨어 및 관련 문서 파일("Software")의 사본을 입수하는 모든 사용자에게 무료로 권한을 부여한다.다음과 같은 조건에 따라 소프트웨어가 제공될 수 있는 이들:

위의 저작권 고지 및 이 허가 고지는 소프트웨어의 모든 사본 또는 상당 부분에 포함되어야 한다.

상품성, 특정 목적에의 적합성 및 비침해의 보증을 포함하되 이에 국한되지 않는 모든 종류의 명시적 또는 묵시적 보증 없이 "있는 그대로 " 소프트웨어는 제공된다. 어떠한 경우에도 저작자나 저작권 소유자는 책임을 지지 않는다. 그렇지 않으면 소프트웨어, 소프트웨어 사용 또는 기타 거래에서 발생하거나, 소프트웨어와 연결되거나, 연결된다.